In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import absolute_import, division, unicode_literals

import sys
import io
import numpy as np
import logging


# Set PATHs
PATH_TO_SENTEVAL = './'
PATH_TO_DATA = './SentEval/data'
PATH_TO_VEC = './data/glove.840B.300d.txt'

# import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval

In [2]:
from nli.preprocess import create_dictionary, get_glove as get_wordvec

[nltk_data] Downloading package punkt to /home/lcur1112/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# SentEval prepare and batcher
def prepare(params, samples):
    _, params.word2id = create_dictionary(samples)
    params.word_vec = get_wordvec(params.word2id, PATH_TO_VEC)
    params.wvec_dim = 300
    return

def batcher(params, batch):
    print('batcher')
    print('params',params)
    print('batch', batch)
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []

    for sent in batch:
        sentvec = []
        for word in sent:
            if word in params.word_vec:
                sentvec.append(params.word_vec[word])
        if not sentvec:
            vec = np.zeros(params.wvec_dim)
            sentvec.append(vec)
        sentvec = np.mean(sentvec, 0)
        embeddings.append(sentvec)

    embeddings = np.vstack(embeddings)
    return embeddings

def batcher(params, batch):

    batch = [sent if sent != [] else ['.'] for sent in batch]
    for sent in batch:
        for word in sent:
            print(word)

    assert False

    return embeddings


# Set params for SentEval
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10}
params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
                                 'tenacity': 3, 'epoch_size': 2}

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks = ['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC',
                      'MRPC', 'SICKEntailment', 'STS14']
    results = se.eval(transfer_tasks)
    print(results)

2023-04-12 21:13:52,596 : ***** Transfer task : MR *****




hello


  5%|▌         | 117355/2196017 [00:03<01:04, 32307.77it/s] 


KeyboardInterrupt: 

## Preprocess again

In [ ]:
import urllib.request
import zipfile
import os
from datasets import load_dataset, load_from_disk
import numpy as np
from tqdm import tqdm
import pickle
import torch
import nltk
nltk.download('punkt')

from nli.preprocess import get_glove

[nltk_data] Downloading package punkt to /home/lcur1112/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/lcur1112/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
dataset_snli = load_from_disk('data/snli')

2023-04-12 20:32:11,082 : open file: /home/lcur1112/NLI/data/snli/dataset_dict.json


In [ ]:
test_samples = dataset_snli['test']['premise'] + dataset_snli['test']['hypothesis']

In [ ]:
id2word, word2id = create_dictionary(test_samples)

# Predict

In [1]:
import torch
import pytorch_lightning as pl
import argparse

from nli.data import NLIDataModule
from nli.models import AvgWordEmb, UniLSTM, BiLSTM, MaxPoolLSTM, MLP, NLINet
from nli.learner import Learner
import os

In [2]:
samples = [['obvious'], ['horrible'], ['crummy'], ['execrable', '.'], ['shallow', '.'], ['eh', '.'], ['abysmally', 'pathetic'], ['terrible', '.'], ['wishy-washy', '.'], ['spiderman', 'rocks'], ['delightfully', 'rendered'], ['surprisingly', 'insightful'], ['touchM-i', '!'], ['refreshing', '.'], ['fantastic', '!'], ['calculated', 'swill', '.'], ['thoroughly', 'awful', '.'], ['truly', 'terrible', '.'], ['by-the-numbers', 'yarn', '.'], ['amazingly', 'dopey', '.'], ['crikey', 'indeed', '.'], ['so-so', 'entertainment', '.'], ['cinematic', 'poo', '.'], ['extremely', 'bad', '.'], ['woefully', 'pretentious', '.'], ['warmed-over', 'hash', '.'], ['lacks', 'depth', '.'], ['under-rehearsed', 'and', 'lifeless'], ['unwieldy', 'contraption', '.'], ['feeble', 'comedy', '.'], ['disjointed', 'parody', '.'], ['two-bit', 'potboiler', '.'], ['painfully', 'padded', '.'], ['dramatically', 'lackluster', '.'], ['incoherence', 'reigns', '.'], ['mildly', 'amusing', '.'], ['fairly', 'run-of-the-mill', '.'], ['mildly', 'entertaining', '.'], ['insufferably', 'naive', '.'], ['amazingly', 'lame', '.'], ['predictably', 'melodramatic', '.'], ['rashomon-for-dipsticks', 'tale', '.'], ['no', 'surprises', '.'], ['thoroughly', 'enjoyable', '.'], ['family', 'fare', '.'], ['compellingly', 'watchable', '.'], ['often', 'hilarious', '.'], ['psychologically', 'savvy', '.'], ['one-of-a-kind', 'near-masterpiece', '.'], ['highly', 'engaging', '.'], ['beautifully', 'produced', '.'], ['psychologically', 'revealing', '.'], ['visually', 'captivating', '.'], ['morvern', 'rocks', '.'], ['genuinely', 'unnerving', '.'], ['deliciously', 'slow', '.'], ['exciting', 'documentary', '.'], ['a', 'muted', 'freak-out'], ['harmless', 'fun', '.'], ['oddly', 'compelling', '.'], ['delirious', 'fun', '.'], ['quietly', 'engaging', '.'], ['a', 'joyous', 'occasion'], ['everything', 'is', 'off', '.'], ['earnest', 'but', 'heavy-handed', '.'], ['one', 'lousy', 'movie', '.'], ['fluffy', 'and', 'disposible', '.'], ['aan', 'opportunity', 'wasted', '.'], ['storytelling', 'feels', 'slight', '.'], ['a', 'high-minded', 'snoozer', '.'], ['banal', 'and', 'predictable', '.'], ['brisk', 'hack', 'job', '.'], ['punitively', 'affirmational', 'parable', '.'], ['decent', 'but', 'dull', '.'], ['thin', 'period', 'piece', '.'], ['well-meant', 'but', 'unoriginal', '.'], ['odd', 'and', 'weird', '.'], ['draggin', "'", 'about', 'dragons'], ['a', 'dreary', 'movie', '.'], ['pompous', 'and', 'garbled', '.'], ['well-meaning', 'but', 'inert', '.'], ['shrewd', 'but', 'pointless', '.'], ['a', 'non-mystery', 'mystery', '.'], ['what', 'an', 'embarrassment', '.'], ['a', 'noble', 'failure', '.'], ['a', 'relative', 'letdown', '.'], ['a', 'puzzling', 'experience', '.'], ['just', 'not', 'campy', 'enough'], ['aggravating', 'and', 'tedious', '.'], ['an', 'awful', 'snooze', '.'], ['just', 'plain', 'silly', '.'], ['a', 'less-than-thrilling', 'thriller', '.'], ['black-and-white', 'and', 'unrealistic', '.'], ['anemic', ',', 'pretentious', '.'], ['i', 'hate', 'this', 'movie'], ['grating', 'and', 'tedious', '.'], ['it', 'bites', 'hard', '.'], ['a', 'pretentious', 'mess', '.'], ['predictably', 'soulless', 'techno-tripe', '.'], ['arty', 'gay', 'film', '.'], ['a', 'half-assed', 'film', '.'], ['bland', 'but', 'harmless', '.'], ['a', 'dreary', 'indulgence', '.'], ['tends', 'to', 'plod', '.'], ['a', 'well-crafted', 'letdown', '.'], ['boring', 'and', 'meandering', '.'], ['less', 'than', 'fresh', '.'], ['a', 'lame', 'comedy', '.'], ['a', 'reality-snubbing', 'hodgepodge', '.'], ['degenerates', 'into', 'hogwash', '.'], ['meandering', 'and', 'confusing', '.'], ['an', 'opportunity', 'missed', '.'], ['inconsequential', 'road-and-buddy', 'pic', '.'], ['a', 'movie', 'to', 'forget'], ['more', 'precious', 'than', 'perspicacious'], ['an', 'intriguing', 'near-miss', '.'], ['bearable', '.', 'barely', '.'], ['staggeringly', 'dreadful', 'romance', '.'], ['well-made', 'but', 'mush-hearted', '.'], ['a', 'real', 'snooze', '.'], ['effective', 'but', 'too-tepid', 'biopic'], ['an', 'imaginative', 'comedy/thriller', '.'], ['an', 'exhilarating', 'experience', '.'], ['troubling', 'and', 'powerful', '.'], ['tailored', 'to', 'entertain', '!'], ['a', 'modest', 'masterpiece', '.'], ['never', 'once', 'predictable', '.'], ['warm', 'and', 'exotic', '.']]

In [3]:
encoder = AvgWordEmb()
classifier = MLP(300*4)

net = NLINet(encoder, classifier)
model = Learner(net)

In [5]:
ckpt_path = 'lightning_logs/version_11564439'
ckpt_path = 'lightning_logs/version_11564821'
ckpts = os.listdir(os.path.join(ckpt_path, 'checkpoints'))
assert len(ckpts) == 1
ckpt_path = os.path.join(ckpt_path, 'checkpoints', ckpts[0])
ckpt_path

'lightning_logs/version_11564821/checkpoints/epoch=8-step=144.ckpt'

In [6]:
model = Learner.load_from_checkpoint(ckpt_path, net=net)
model.eval()

Learner(
  (net): NLINet(
    (encoder): AvgWordEmb()
    (classifier): MLP(
      (mlp): Sequential(
        (0): Linear(in_features=1200, out_features=512, bias=True)
        (1): Linear(in_features=512, out_features=512, bias=True)
        (2): Linear(in_features=512, out_features=3, bias=True)
      )
    )
  )
  (loss_fn): CrossEntropyLoss()
)

In [7]:
model.encode()

NotImplementedError: Module [Learner] is missing the required "forward" function

'lightning_logs/version_11564821/checkpoints/epoch=8-step=144.ckpt'

In [52]:
checkpoint = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model.load_state_dict(checkpoint['state_dict'])


RuntimeError: Error(s) in loading state_dict for Learner:
	Missing key(s) in state_dict: "net.encoder.lstm.weight_ih_l0", "net.encoder.lstm.weight_hh_l0", "net.encoder.lstm.bias_ih_l0", "net.encoder.lstm.bias_hh_l0", "net.encoder.lstm.weight_ih_l0_reverse", "net.encoder.lstm.weight_hh_l0_reverse", "net.encoder.lstm.bias_ih_l0_reverse", "net.encoder.lstm.bias_hh_l0_reverse". 
	size mismatch for net.classifier.mlp.0.weight: copying a param with shape torch.Size([512, 1200]) from checkpoint, the shape in current model is torch.Size([512, 16384]).

In [32]:
model

nli.learner.Learner

TypeError: Learner.__init__() missing 1 required positional argument: 'net'